In [1]:
import pandas as pd
pd.__version__

'2.2.3'

In [2]:
url = 'https://raw.githubusercontent.com/frandiego/cei/refs/heads/main/data/penguins.csv'

In [3]:
# read data
df = pd.read_csv(url)
df.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE


In [4]:
# columns in lowercase -> yes
df.columns

Index(['species', 'island', 'bill_length_mm', 'bill_depth_mm',
       'flipper_length_mm', 'body_mass_g', 'sex'],
      dtype='object')

In [5]:
# any missing data? -> yes [sex, bill_length_mm, bill_depth_mm, flipper_length_mm]
df.isna().sum()

species               0
island                0
bill_length_mm        2
bill_depth_mm         2
flipper_length_mm     2
body_mass_g           2
sex                  11
dtype: int64

In [6]:
# function to fill sex
def fill_sex(row): 
    if not pd.isna(row['sex']):
        return row['sex'].lower()
    elif not pd.isna(row['body_mass_g']):
        if row['body_mass_g']< 4225:
            return 'female'
        return 'male'
    elif row['island'].lower() == 'torgersen':
        return 'female'
    return 'male'

In [7]:
# fill sex using function and apply
# apply will allow us to apply the function
# all over the rows
df['sex'] = df.apply(fill_sex, axis=1)
df.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,female
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female


In [8]:
# separate string and numeric columns
string_columns = list(df.select_dtypes('object').columns)
number_columns = list(df.select_dtypes('number').columns)
print('string_columns: ', string_columns)
print('number_columns: ', number_columns)

string_columns:  ['species', 'island', 'sex']
number_columns:  ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']


In [9]:
# fill number columns by using a group by over string columns
for col in number_columns:
    fill_serie = df.groupby(string_columns)[col].transform('median')
    df[col] = df[col].fillna(fill_serie)

In [10]:
# check there is no missing data
df.isna().sum()

species              0
island               0
bill_length_mm       0
bill_depth_mm        0
flipper_length_mm    0
body_mass_g          0
sex                  0
dtype: int64

In [11]:
# create a dictionary to map string to integer
mapper = {}
for col in string_columns:
    unique_categories = sorted(df[col].unique())
    categories_integer = list(range(1, len(df[col].unique())+1))
    mapper[col] = dict(zip(unique_categories, categories_integer))
mapper

{'species': {'Adelie': 1, 'Chinstrap': 2, 'Gentoo': 3},
 'island': {'Biscoe': 1, 'Dream': 2, 'Torgersen': 3},
 'sex': {'female': 1, 'male': 2}}

In [12]:
# change string to integer using mapper
for colname, map_categories in mapper.items():
    df[colname] = df[colname].map(map_categories)
df.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,1,3,39.1,18.7,181.0,3750.0,2
1,1,3,39.5,17.4,186.0,3800.0,1
2,1,3,40.3,18.0,195.0,3250.0,1
3,1,3,37.8,17.4,189.0,3450.0,1
4,1,3,36.7,19.3,193.0,3450.0,1


In [13]:
# create a column is_female 
# boolean column to check if sex is female
df['is_female'] = df['sex'] == 1
# from boolean to integer
df['is_female'] = df['is_female'].astype(int)
# remove sex column, now is redundant
df = df.drop(['sex'], axis=1)

In [14]:
df.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,is_female
0,1,3,39.1,18.7,181.0,3750.0,0
1,1,3,39.5,17.4,186.0,3800.0,1
2,1,3,40.3,18.0,195.0,3250.0,1
3,1,3,37.8,17.4,189.0,3450.0,1
4,1,3,36.7,19.3,193.0,3450.0,1


In [18]:
# store as csv
df.to_csv('../data/penguins_cleaned.csv', index=False)